# Mathematical Image Analysis Final Project: Dataset Augmentation

## import libraries

In [ ]:
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import numpy as np

## import sample image

In [ ]:
# Load the image using OpenCV (note: this loads in BGR format)
img = cv2.imread('motorcycle.jpg')
# Convert BGR to RGB for display
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title('Motorcycle Image')
plt.axis('on')
plt.show()

# get image as numpy array
img_array = np.array(img_rgb)

# Point operations (I.4):  

In [ ]:
# changing exposure/brightness {brightening (1.5)}


In [ ]:
# contrast enhancement (I.6)


In [ ]:
# enhancing saturation

# converting to black and white

In [ ]:
# Introducing random salt and pepper noise


# Filtering using convolution (I.7, 15)

In [ ]:
def convolution2d(image, kernel, stride=1):
    """
    Helper function to perform a 2D convolution on an image with a given kernel.
    Parameters:
    - image: 2D numpy array representing the input image
    - kernel: 2D numpy array representing the convolution kernel
    - stride: integer, the step size for the convolution (1)
    Returns:
    - output: 2D numpy array representing the convolved image
    """
    
    # get dimensions
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # pads to keep output the same size as input
    pad_h = (kernel_height - 1) // 2
    pad_w = (kernel_width - 1) // 2

    # output array
    output_height = (image_height - kernel_height + 2 * pad_h) // stride + 1
    output_width = (image_width - kernel_width + 2 * pad_w) // stride + 1

    output = np.zeros((output_height, output_width))
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')

    # convolve image and filter
    for y in range(output_height):
        for x in range(output_width):
            region = padded_image[y * stride:y * stride + kernel_height,
                                  x * stride:x * stride + kernel_width]
            output[y, x] = np.sum(region * kernel)

    return output

In [ ]:
# Gaussian Blur (example of weighted averaging filter (I.12))
def gaussian_blur(image, kernel_size=9, sigma=1.0):
    """
    Apply Gaussian blur to an image using a Gaussian kernel (handles RGB images).
    Parameters:
    - image: 2D numpy array representing the input image
    - kernel_size: size of the Gaussian kernel (must be odd)
    - sigma: standard deviation of the Gaussian distribution
    Returns:
    - blurred_image: 2D numpy array representing the blurred image
    """
    
    # set up the kernel
    gaussian_blur_kernel = np.zeros((kernel_size, kernel_size), dtype=np.float32)
    center = kernel_size // 2
    for x in range(kernel_size):
        for y in range(kernel_size):
            gaussian_blur_kernel[x, y] = (1 / (2 * np.pi * sigma ** 2)) * \
                np.exp(-((x - center) ** 2 + (y - center) ** 2) / (2 * sigma ** 2))
    gaussian_blur_kernel /= np.sum(gaussian_blur_kernel)

    # handle grayscale and RGB images
    if image.ndim == 2:
        return convolution2d(image, gaussian_blur_kernel)
    elif image.ndim == 3:
        blurred_channels = [
            convolution2d(image[:, :, c], gaussian_blur_kernel)
            for c in range(image.shape[2])
        ]
        return np.stack(blurred_channels, axis=-1)

In [ ]:
# test the Gaussian blur function
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(img_rgb)
plt.title("Original")

plt.subplot(1, 3, 2)
plt.imshow(cv2.GaussianBlur(img_rgb, (9, 9), sigmaX=1.0))
plt.title("OpenCV Blur")

plt.subplot(1, 3, 3)
plt.imshow(gaussian_blur(img_rgb, kernel_size=9, sigma=1.0).astype(np.uint8))
plt.title("Manual Blur")

plt.tight_layout()
plt.show()


In [ ]:
# Sharpen the image (I.55-59)
def sharpen_image(image):
    """
    Sharpen the image using a Laplacian kernel.
    Parameters:
    - image: 2D numpy array representing the input image
    Returns:
    - sharpened_image: 2D numpy array representing the sharpened image
    """
    # define kernel
    laplacian_kernel = np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])
    
    # apply convolution to sharpen the image
    sharpened_image = convolution2d(image, laplacian_kernel)
    
    return sharpened_image

# Other (from linear algebra review):


In [ ]:
# Fixed/Random rotation

In [ ]:
# Shearing; typically padded with 0s (I.9), change in basis

In [ ]:
# Flipping image (reflection) 